<a href="https://colab.research.google.com/github/FedorTaggenbrock/data_intensive_systems/blob/main/notebooks/main_tests_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python ../main_tests.py

python3: can't open file '/content/../main_tests.py': [Errno 2] No such file or directory


**Pip install stuff that needs to be installed before importing anything**  
(for colab only; cannot be done inside the modules, would be stupid)

In [1]:
import sys
ON_COLAB = 'google.colab' in sys.modules
if ON_COLAB:
    # Do stuff that only needs to happen on colab
    !pip install pyspark
else:
    # Do stuff that only needs to happen on local computer
    pass

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=bd3670f4bd30b47807938dcc60859d6d40e0ee181448346a9a74ab877a8cde2f
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [ ]:
if ON_COLAB:
    print("Running in Colab: importing from GitHub.")
    # wget ("curl") all files from github
    user = 'FedorTaggenbrock'
    repo = 'data_intensive_systems'
    src_dir = "src"
    pyfiles = ["clustering.py"]

    for pyfile in pyfiles:
        url = f"https://raw.githubusercontent.com/{user}/{repo}/main/{src_dir}/{pyfile}"
        !wget --no-cache --backups=1 {url}

    # Import all files
    try:
        import clustering as clustering
    except Exception as e:
        print('Error importing files on colab. Are the names right?.')

else:
    print("Running locally, importing from local.")
    # Append parent folder to path
    sys.path.append('..')
    # And import all files
    from src import clustering as clustering


In [4]:
import os
import requests
import sys

def get_file_from_github(raw_url, target_file):
    response = requests.get(raw_url)
    response.raise_for_status()  # Ensure we got a valid response
    with open(target_file, 'wb') as out_file:
        out_file.write(response.content)

def import_from_github(raw_urls, module_names):
    # Set your target path here
    # In Google Colab, you can use the current directory
    target_path = os.getcwd()

    # Download files
    for raw_url, module_name in zip(raw_urls, module_names):
        target_file = f"{target_path}/{module_name}.py"
        if not os.path.exists(target_file):
            get_file_from_github(raw_url, target_file)

    # Add the directory to sys.path
    sys.path.insert(0, target_path)

    # Now you can import your modules
    modules = [__import__(module_name) for module_name in module_names]
    
    return modules


user = 'FedorTaggenbrock'
repo = 'data_intensive_systems'
src_dir = "src"
module_names = ["clustering", 'main_tests']
raw_urls = [f"https://raw.githubusercontent.com/{user}/{repo}/main/{src_dir}/{modname}.py" for modname in module_names]

# github_url = 'https://github.com/FedorTaggenbrock/data_intensive_systems.git'

clustering, main_tests = import_from_github(raw_urls, module_names)

# Now you can use your modules
res = clustering.jaccard_distance([1,1,0,1,0], [1,1,1,1,0])
print(res)
clustering.clustering_test1()

0.25
update 13:30
Initialized Spark. Start clustering.
centroids =  [(1, 0, 0, 1, 0), (1, 1, 1, 1, 0)]
centroids =  [(1, 0, 0, 1, 0), (1, 1, 1, 1, 0)]
Finished clustering. Start evaluation.
done


['ArrayType',
 'List',
 'RDD',
 'SparkContext',
 'SparkSession',
 'StringType',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'clustering_test1',
 'evaluate_clustering',
 'evaluate_kModes',
 'jaccard_distance',
 'kModes_v2',
 'mode',
 'np',
 'scipy',
 'udf']

**Test if this works**


In [7]:
# print(dir(clustering))
res = clustering.jaccard_distance([1,1,0,1,0], [1,1,1,1,0])
print(res)
clustering.clustering_test1()

0.25
Initialized Spark. Start clustering.
centroids =  [(1, 0, 0, 1, 0), (1, 0, 0, 0, 1)]
centroids =  [(1, 1, 0, 1, 0), (0, 0, 1, 0, 1)]
Finished clustering. Start evaluation.
done


**Create a spark instance**

In [ ]:
spark_settings = {}
spark_instance = make_spark()

**Load and parse data**

In [ ]:
# Load and parse data
data_for_spark = parse_data()

In [ ]:
# Run clustering and parameter tuning
clustering_settings = {}
clustering_centroid_outcomes = run_clustering(spark_instance, data_for_spark, clustering_settings)

In [ ]:
# Evaluate clustering results
clustering_evaluation = evaluate_clustering(clustering_centroid_outcomes)

In [ ]:
# Display results (maybe only best result)
clustering_result = display_results(clustering_evaluation